## ORF News --> Reddit Tracking Pipeline (InfluxDB)

## Ziel der Analyse
Diese Analyse sammelt aktuelle ORF-Nachrichten,
filtert politische Auslandsartikel,
speichert sie in InfluxDB
und sucht anschließend passende Diskussionen auf Reddit.

In [ ]:
%pip install requests feedparser
%pip install pandas
%pip install googletrans==4.0.0-rc1
%pip install influxdb-client

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Schritt 1 – Abruf des ORF RSS Feeds

Die ORF-Nachrichten werden über den offiziellen RSS-Feed geladen.
Vorteile:

- kein HTML-Scraping
- strukturierte Metadaten
- stabile Schnittstelle

Der Feed wird bewusst **roh (XML)** geladen,
damit wir volle Kontrolle über Parsing und Filterlogik haben.


In [ ]:
# Cell 1 - imports + config (nur DB)
import os
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timezone

# InfluxDB (lesen von env)
INFLUX_URL = os.getenv("INFLUX_URL", "http://localhost:8086")
INFLUX_TOKEN = os.getenv("INFLUX_TOKEN", "")
INFLUX_ORG = os.getenv("INFLUX_ORG", "")
INFLUX_BUCKET = os.getenv("INFLUX_BUCKET", "")

FEED_URL = "https://rss.orf.at/news.xml"

TARGET_OEWA = "urn:oewa:RedCont:Politik/PolitikAusland"
USER_AGENT = "orf-rss-tracker/1.0 (+local notebook)"


## Schritt 2 – Dynamische Namespace-Erkennung

Der ORF RSS Feed nutzt mehrere XML-Namespaces
(z.B. RSS, RDF, Dublin Core, ORF-spezifisch).

Statt diese hart zu codieren,
werden sie automatisch aus dem Dokument erkannt.

Pro:
- robust gegen Feed-Änderungen
- vermeidet klassische XML-Parsing-Fehler


In [ ]:
# Cell 2 - fetch XML
def fetch_feed_xml(url: str, timeout: int = 20) -> str:
    r = requests.get(url, timeout=timeout, headers={"User-Agent": USER_AGENT})
    r.raise_for_status()
    return r.text

xml_text = fetch_feed_xml(FEED_URL)
len(xml_text), xml_text[:200]


(13907,
 '<?xml version="1.0" encoding="UTF-8"?>\n<rdf:RDF\n  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n  xmlns:dc="http://purl.org/dc/elements/1.1/"\n  xmlns:sy="http://purl.org/rss/1.0/modules/synd')

## Schritt 3 – Sanity Checks

Bevor Daten weiterverarbeitet werden, prüfen wir:

- Root-Element des XML
- Anzahl der gefundenen Items
- ob der Feed überhaupt Inhalte liefert

Diese Checks erklären auch, warum frühere Versionen keine Daten geliefert haben.


In [ ]:
# Cell 3 - detect namespaces robustly (so you don't have to guess)
import io

def detect_namespaces(xml_text: str) -> dict:
    ns = {}
    for event, elem in ET.iterparse(io.StringIO(xml_text), events=("start-ns",)):
        prefix, uri = elem
        ns[prefix if prefix is not None else ""] = uri
    return ns

NS = detect_namespaces(xml_text)
NS


{'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'dc': 'http://purl.org/dc/elements/1.1/',
 'sy': 'http://purl.org/rss/1.0/modules/syndication/',
 'orfon': 'http://rss.orf.at/1.0/',
 '': 'http://purl.org/rss/1.0/'}

In [ ]:
# Cell 4 - parse + sanity checks (THIS will show why your old code returned 0)
# Cell 4 - parsen + sannity checks (wird zeigen, warum der alte Code 0 zurückgegeben hat)
root = ET.fromstring(xml_text)

rss_ns = NS.get("rss", "http://purl.org/rss/1.0/")  # ORF uses RSS 1.0
items = root.findall(".//{%s}item" % rss_ns)

root.tag, len(items)


('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF', 21)

## Schritt 4 – Deduplizierung (InfluxDB)

Um doppelte Einträge zu vermeiden:

- vorhandene ORF-Artikel werden aus InfluxDB geladen
- USIDs der letzten 30 Tage werden zwischengespeichert
- nur **neue Artikel** werden weiterverarbeitet

InfluxDB dient hier nicht nur als Speicher, sondern auch als **Gedächtnis der Pipeline**.


In [ ]:
# Cell 5 - helper: load already-seen usids (dedupe) from InfluxDB (DB-only)
# Cell 5 - helfer laden bereits gesehener usids (dedupe) von InfluxDB (nur DB)
from influx_io import get_client, INFLUX_BUCKET, INFLUX_ORG

def load_seen_usids_from_influx(lookback: str = "30d") -> set[str]:
    """
    Pull distinct usid tag values stored in measurement 'orf_article'
    within a recent lookback window.
    """
    with get_client() as client:
        query_api = client.query_api()

        flux = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: -{lookback})
  |> filter(fn: (r) => r._measurement == "orf_article")
  |> keep(columns: ["usid"])
  |> distinct(column: "usid")
'''

        tables = query_api.query(flux, org=INFLUX_ORG)

    seen = set()
    for table in tables:
        for record in table.records:
            val = record.values.get("usid")
            if val:
                seen.add(str(val))
    return seen

seen_usids = load_seen_usids_from_influx("30d")
len(seen_usids)


19

## Schritt 5 – Fachliches Filtern (OEWA-Kategorie)

Nicht alle ORF-Artikel sind relevant.
Gefiltert wird gezielt nach:

- OEWA-Kategorie: *Politik Ausland*

Technisch:
- `orfon:oewaCategory`
- Vergleich über RDF-Resource-Attribut

Das stellt sicher, dass nur inhaltlich relevante Artikel in die Analyse gelangen.


In [ ]:
# Cell 6 - parse items + filter by oewaCategory
# Cell 6 - parsen items + filtern nach oewaCategory
def text_of(el):
    return el.text.strip() if el is not None and el.text else None

def parse_filtered_items(root: ET.Element, ns: dict) -> list[dict]:
    rss_ns = ns.get("rss", "http://purl.org/rss/1.0/")
    rdf_ns = ns.get("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    dc_ns  = ns.get("dc",  "http://purl.org/dc/elements/1.1/")
    orf_ns = ns.get("orfon")  # must exist in feed; we'll rely on detected value / muss im Feed existieren; wir verlassen uns auf den erkannten Wert

    if not orf_ns:
        raise RuntimeError("Could not detect 'orfon' namespace in the feed. Check NS dict output.")

    out = []
    for item in root.findall(".//{%s}item" % rss_ns):
        # orfon:oewaCategory rdf:resource="..."
        cat_el = item.find("{%s}oewaCategory" % orf_ns)
        if cat_el is None:
            continue

        cat_val = cat_el.attrib.get("{%s}resource" % rdf_ns)
        if cat_val != TARGET_OEWA:
            continue

        title_el = item.find("{%s}title" % rss_ns)
        link_el  = item.find("{%s}link" % rss_ns)
        date_el  = item.find("{%s}date" % dc_ns)
        usid_el  = item.find("{%s}usid" % orf_ns)

        out.append({
            "usid": text_of(usid_el),
            "date": text_of(date_el),
            "link": text_of(link_el),
            "title": text_of(title_el),
            "oewaCategory": cat_val,
            "fetched_at_utc": datetime.now(timezone.utc).isoformat(),
        })

    return out

filtered_items = parse_filtered_items(root, NS)
len(filtered_items), filtered_items[:2]


(11,
 [{'usid': 'news:3417010',
   'date': '2026-01-13T12:49:06+01:00',
   'link': 'https://orf.at/stories/3417010/',
   'title': 'Ungarns Parlamentspräsident attackiert von der Leyen',
   'oewaCategory': 'urn:oewa:RedCont:Politik/PolitikAusland',
   'fetched_at_utc': '2026-01-13T12:36:15.786761+00:00'},
  {'usid': 'news:3417006',
   'date': '2026-01-13T12:35:39+01:00',
   'link': 'https://orf.at/stories/3417006/',
   'title': 'Staatliche Angabe: Rund 2.000 Tote bei Protesten im Iran',
   'oewaCategory': 'urn:oewa:RedCont:Politik/PolitikAusland',
   'fetched_at_utc': '2026-01-13T12:36:15.786761+00:00'}])

In [ ]:
# Cell 7 - (optional) DEBUG: what categories exist + counts
# Cell 7 - (optional) DEBUG: welche Kategorien existieren + Zählungen
from collections import Counter

def category_counts(root: ET.Element, ns: dict) -> Counter:
    rss_ns = ns.get("rss", "http://purl.org/rss/1.0/")
    rdf_ns = ns.get("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    orf_ns = ns.get("orfon")
    c = Counter()
    for item in root.findall(".//{%s}item" % rss_ns):
        cat_el = item.find("{%s}oewaCategory" % orf_ns) if orf_ns else None
        if cat_el is None:
            continue
        cat_val = cat_el.attrib.get("{%s}resource" % rdf_ns)
        if cat_val:
            c[cat_val] += 1
    return c

counts = category_counts(root, NS)
counts.most_common(10)


[('urn:oewa:RedCont:Politik/PolitikAusland', 11),
 ('urn:oewa:RedCont:Nachrichten/Chronik', 4),
 ('urn:oewa:RedCont:KulturUndFreizeit/KulturUeberblick', 3),
 ('urn:oewa:RedCont:Wirtschaft/Wirtschaftspolitik', 2),
 ('urn:oewa:RedCont:Politik/PolitikInland', 1)]

## Schritt 6 – Schreiben der ORF-Artikel in InfluxDB

Jeder ORF-Artikel wird als Time-Series-Datenpunkt gespeichert:

Measurement:
- `orf_article`

Tags:
- `usid`
- `category`

Fields:
- `title`
- `link`

Time:
- Veröffentlichungsdatum (UTC)

Dadurch sind spätere Zeitreihen-Visualisierungen möglich.


In [ ]:
# Cell 8 - DB-only: keep only new items + write to InfluxDB
# Cell 8 - DB-only: nur neue Items behalten + in InfluxDB schreiben
from influx_io import write_orf_articles

new_items = [it for it in filtered_items if it.get("usid") and it["usid"] not in seen_usids]

written = write_orf_articles(new_items)
print("Wrote ORF articles to InfluxDB:", written)

# update local seen set (so rerunning later cells in this same session doesn't re-write)
# updated local gesehenes Set (damit spätere Zellen in derselben Sitzung nicht erneut geschrieben werden)
for it in new_items:
    seen_usids.add(it["usid"])

len(new_items)


Wrote ORF articles to InfluxDB: 1


1

In [ ]:
# Cell 9 - show latest rows quickly (DB-only, from current run)
# Cell 9 - zeige neueste Zeilen schnell an (nur DB, aus dem aktuellen Lauf)
import pandas as pd

df = pd.DataFrame(new_items)
df.tail(10)


,usid,date,link,title,oewaCategory,fetched_at_utc
0,news:3417010,2026-01-13T12:49:06+01:00,https://orf.at/stories/3417010/,Ungarns Parlamentspräsident attackiert von der...,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-13T12:36:15.786761+00:00


# Reddit

## Schritt 7 – Reddit-Suche zu ORF-Artikeln

Für jeden ORF-Artikel werden passende Reddit-Posts gesucht.

Strategie:
- mehrere Suchqueries (URL, Domain, Titel)
- Pagination über Reddit Search API
- heuristische Filterung (Keyword-Overlap)

Ziel:
- mindestens 20 relevante Posts pro Artikel


## Schritt 8 – Heuristische Relevanzprüfung

Ein Reddit-Post wird nur übernommen, wenn:

- ORF-Link oder Domain vorkommt
ODER
- ausreichende Wort-Überschneidung mit dem ORF-Titel existiert

Diese Heuristik reduziert False Positives
ohne NLP-Overkill.


## Schritt 9 – Schreiben der Reddit-Posts in InfluxDB

Measurement:
- `reddit_post`

Tags:
- `usid` (ORF-Artikel)
- `source` (Subreddit)

Fields:
- Titel, URLs, Text
- Wortanzahl
- Matching-Metriken

Time:
- Erstellungszeit des Reddit-Posts (UTC)

ORF-Artikel und Reddit-Posts sind über `usid` verknüpft.


In [ ]:

# Cell X - Reddit: find related posts for ORF articles and write to InfluxDB
# Notes:
# - Uses the public Reddit JSON search endpoint (no auth). This is rate-limited; be gentle.
# - Tries multiple queries per article (title, url/domain) and paginates until it has >=20 unique posts.
# - Writes measurement "reddit_post" with tags/fields as requested.

# Cell X - Reddit: finde verwandte Beiträge für ORF-Artikel und schreibe in InfluxDB
# Hinweise:
# - Verwendet den öffentlichen Reddit JSON-Suchendpunkt (keine Authentifizierung). Dies ist rate-limitiert; seien Sie sanft.
# - Versucht mehrere Abfragen pro Artikel (Titel, URL/Domain) und paginiert, bis es >=20 eindeutige Beiträge hat.
# - Schreibt die Messung "reddit_post" mit den angeforderten Tags/Feldern.

import time
import re
from datetime import datetime, timezone

import requests
from influxdb_client import Point

# ---- Config ----
REDDIT_UA = os.getenv("REDDIT_USER_AGENT", "orf-reddit-matcher/1.0 (contact: local)")
REDDIT_SLEEP_S = float(os.getenv("REDDIT_SLEEP_S", "1.2"))  # be kind
REDDIT_TIMEOUT = int(os.getenv("REDDIT_TIMEOUT", "20"))
REDDIT_MAX_PAGES_PER_QUERY = int(os.getenv("REDDIT_MAX_PAGES_PER_QUERY", "6"))  # 6*100=600 max per query
REDDIT_MIN_POSTS_PER_ARTICLE = int(os.getenv("REDDIT_MIN_POSTS_PER_ARTICLE", "20"))
REDDIT_LOOKBACK = os.getenv("REDDIT_LOOKBACK", "all")  # "all", "year", "month", ...

# ---- Helpers ----
def _clip(s: str, n: int) -> str:
    if s is None:
        return ""
    s = str(s)
    return s if len(s) <= n else s[:n-1] + "…"

def _words(s: str) -> int:
    if not s:
        return 0
    return len(re.findall(r"\b\w+\b", s))

def _reddit_search(query: str, after: str | None = None, limit: int = 100, sort: str = "relevance", t: str = "all") -> dict:
    """Calls Reddit's public search endpoint and returns parsed JSON."""
    base = "https://www.reddit.com/search.json"
    params = {
        "q": query,
        "limit": str(limit),
        "sort": sort,
        "t": t,
        "restrict_sr": "false",
        "include_over_18": "on",
        "type": "link",
        "raw_json": "1",
    }
    if after:
        params["after"] = after

    r = requests.get(base, params=params, timeout=REDDIT_TIMEOUT, headers={"User-Agent": REDDIT_UA})
    if r.status_code == 429:
        time.sleep(max(5.0, REDDIT_SLEEP_S * 4))
        r = requests.get(base, params=params, timeout=REDDIT_TIMEOUT, headers={"User-Agent": REDDIT_UA})
    r.raise_for_status()
    return r.json()

def _article_queries(title: str, url: str | None) -> list[str]:
    """Build a small set of queries (URL/domain + title-based)."""
    qs = []
    if url:
        qs.append(f'url:"{url}"')
        qs.append("site:orf.at " + " ".join(re.findall(r"\w+", title)[:8]))
    if title:
        qs.append(f'"{title}"')
        qs.append(" ".join(re.findall(r"\w+", title)[:10]) + " orf")
    seen = set()
    out = []
    for q in qs:
        q2 = q.strip()
        if q2 and q2 not in seen:
            out.append(q2)
            seen.add(q2)
    return out

def find_reddit_posts_for_article(usid: str, title: str, url: str | None) -> list[dict]:
    """Return list of dicts with keys expected by the Point(...) writer."""
    results = {}
    queries = _article_queries(title, url)

    for q in queries:
        after = None
        for _page in range(REDDIT_MAX_PAGES_PER_QUERY):
            data = _reddit_search(q, after=after, limit=100, sort="relevance", t=REDDIT_LOOKBACK)
            children = (data.get("data") or {}).get("children") or []
            after = (data.get("data") or {}).get("after")

            for ch in children:
                d = (ch.get("data") or {})
                reddit_id = d.get("id")
                if not reddit_id or reddit_id in results:
                    continue

                permalink = "https://www.reddit.com" + str(d.get("permalink") or "")
                post_url = str(d.get("url") or "")
                selftext = str(d.get("selftext") or "")
                title_r = str(d.get("title") or "")

                hay = (title_r + " " + selftext + " " + post_url).lower()

                keep = False
                if url and url.lower() in hay:
                    keep = True
                elif "orf.at" in hay:
                    keep = True
                else:
                    toks = [t.lower() for t in re.findall(r"\w+", title) if len(t) >= 5][:8]
                    overlap = sum(1 for t in toks if t in hay)
                    keep = overlap >= 2  # heuristic

                if not keep:
                    continue

                results[reddit_id] = {
                    "usid": usid,
                    "source": d.get("subreddit") or "",
                    "reddit_id": reddit_id,
                    "reddit_title": title_r,
                    "reddit_permalink": permalink,
                    "post_url": post_url,
                    "reddit_selftext": selftext,
                    "checked_word_count": _words(title_r) + _words(selftext),
                    "group_matches_in_window": 0,
                    "created_utc": d.get("created_utc"),
                }

            time.sleep(REDDIT_SLEEP_S)

            if len(results) >= REDDIT_MIN_POSTS_PER_ARTICLE:
                break
            if not after:
                break

        if len(results) >= REDDIT_MIN_POSTS_PER_ARTICLE:
            break

    out = list(results.values())
    out.sort(key=lambda r: (r.get("created_utc") or 0), reverse=True)
    return out[:REDDIT_MIN_POSTS_PER_ARTICLE]

# ---- Load ORF articles von InfluxDB ----
from influx_io import get_client, INFLUX_BUCKET, INFLUX_ORG

def load_orf_articles_from_influx(lookback: str = "30d") -> list[dict]:
    """Pull ORF articles (measurement 'orf_article') within lookback."""
    with get_client() as client:
        query_api = client.query_api()
        flux = f"""
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: -{lookback})
  |> filter(fn: (r) => r._measurement == "orf_article")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "usid", "title", "link", "date"])
"""
        tables = query_api.query(flux, org=INFLUX_ORG)

    rows = []
    for t in tables:
        for rec in t.records:
            d = rec.values
            rows.append({
                "time": d.get("_time"),
                "usid": d.get("usid"),
                "title": d.get("title"),
                "link": d.get("link"),
                "date": d.get("date"),
            })

    dedup = {}
    for r in rows:
        u = r.get("usid")
        if u and u not in dedup:
            dedup[u] = r
    return list(dedup.values())

articles = load_orf_articles_from_influx(lookback="30d")
print("Loaded ORF articles:", len(articles))
articles[:2]

# ---- Write reddit posts to InfluxDB ----
# ---- Schreibe reddit Beiträge in InfluxDB ----
def write_reddit_posts(rows: list[dict]) -> int:
    if not rows:
        return 0
    with get_client() as client:
        write_api = client.write_api()
        points = []
        for r in rows:
            usid = str(r.get("usid") or "")
            reddit_id = str(r.get("reddit_id") or "")
            created_utc = r.get("created_utc")
            if created_utc:
                dt = datetime.fromtimestamp(float(created_utc), tz=timezone.utc)
            else:
                dt = datetime.now(timezone.utc)

            points.append(
                Point("reddit_post")
                .tag("usid", usid)
                .tag("source", str(r.get("source") or ""))
                .field("reddit_id", reddit_id)
                .field("title", str(r.get("reddit_title") or ""))
                .field("permalink", str(r.get("reddit_permalink") or ""))
                .field("url", str(r.get("post_url") or ""))
                .field("checked_word_count", int(r.get("checked_word_count") or 0))
                .field("group_matches_in_window", int(r.get("group_matches_in_window") or 0))
                .field("selftext", _clip(str(r.get("reddit_selftext") or ""), 8000))
                .time(dt)
            )

        write_api.write(bucket=INFLUX_BUCKET, org=INFLUX_ORG, record=points)
    return len(points)

# ---- Main loop ----
all_written = 0
for a in articles:
    usid = a.get("usid")
    title = a.get("title") or ""
    link = a.get("link")
    if not usid or not title:
        continue

    reddit_rows = find_reddit_posts_for_article(usid=usid, title=title, url=link)
    if len(reddit_rows) < REDDIT_MIN_POSTS_PER_ARTICLE:
        print(f"[WARN] usid={usid}: found only {len(reddit_rows)} reddit posts for '{title[:70]}…'")
    w = write_reddit_posts(reddit_rows)
    all_written += w
    print(f"usid={usid}: wrote {w} reddit posts")
print("Total reddit posts written:", all_written)


Loaded ORF articles: 20
[WARN] usid=news:3416894: found only 5 reddit posts for 'Chinas Boykott treibt Japan in Tabubruch…'
usid=news:3416894: wrote 5 reddit posts
usid=news:3416950: wrote 20 reddit posts
usid=news:3416953: wrote 20 reddit posts
usid=news:3416954: wrote 20 reddit posts
usid=news:3416956: wrote 20 reddit posts
usid=news:3416957: wrote 20 reddit posts
usid=news:3416959: wrote 20 reddit posts
usid=news:3416962: wrote 20 reddit posts
usid=news:3416964: wrote 20 reddit posts
usid=news:3416966: wrote 20 reddit posts
usid=news:3416967: wrote 20 reddit posts
usid=news:3416968: wrote 20 reddit posts
[WARN] usid=news:3416987: found only 12 reddit posts for '„Kriegsverbrechen“ mit getarntem Flugzeug…'
usid=news:3416987: wrote 12 reddit posts
[WARN] usid=news:3416989: found only 14 reddit posts for 'Gewalt im Iran: Spanien und Finnland bestellen Botschafter ein…'
usid=news:3416989: wrote 14 reddit posts
[WARN] usid=news:3416990: found only 14 reddit posts for 'Iranische Justiz kla

## Schritt 10 – Inspection der gespeicherten Daten

Zum Abschluss:
- Laden der Reddit-Posts aus InfluxDB
- Gruppierung nach ORF-Artikel
- Anzeige der jeweils 20 gespeicherten Posts

Dies dient:
- Debugging
- Qualitätssicherung
- Vorbereitung für Visualisierungen


In [ ]:
# Cell Y - Inspect: show the 20 reddit posts per ORF article (read back from InfluxDB)
# Fixes the Pandas "_time" attribute issue by using row["_time"] via iterrows().

# Cell Y - Inspect: zeige die 20 reddit Beiträge pro ORF-Artikel (lies zurück von InfluxDB)
# Behebt das Pandas "_time"-Attributproblem, indem row["_time"] über iterrows() verwendet wird.

import pandas as pd
from influx_io import get_client, INFLUX_BUCKET, INFLUX_ORG

def load_reddit_posts_from_influx(lookback: str = "30d") -> pd.DataFrame:
    """
    Loads measurement 'reddit_post' from InfluxDB for inspection.
    reddit_id/title/permalink/url/selftext are written as FIELDS; usid/source are tags.
    """
    with get_client() as client:
        query_api = client.query_api()
        flux = f"""
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: -{lookback})
  |> filter(fn: (r) => r._measurement == "reddit_post")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time","usid","source","reddit_id","title","permalink","url","checked_word_count","group_matches_in_window","selftext"])
"""
        tables = query_api.query(flux, org=INFLUX_ORG)

    rows = []
    for t in tables:
        for rec in t.records:
            v = rec.values
            rows.append({
                "_time": v.get("_time"),
                "usid": v.get("usid"),
                "source": v.get("source"),
                "reddit_id": v.get("reddit_id"),
                "title": v.get("title"),
                "permalink": v.get("permalink"),
                "url": v.get("url"),
                "checked_word_count": v.get("checked_word_count"),
                "group_matches_in_window": v.get("group_matches_in_window"),
                "selftext": v.get("selftext"),
            })

    df = pd.DataFrame(rows)
    if df.empty:
        print("No reddit_post points found in Influx for lookback =", lookback)
        return df

    # normalize numeric columns
    # normalisiere numerische Spalten
    df["checked_word_count"] = pd.to_numeric(df["checked_word_count"], errors="coerce").fillna(0).astype(int)
    df["group_matches_in_window"] = pd.to_numeric(df["group_matches_in_window"], errors="coerce").fillna(0).astype(int)

    # sort: most recent first within each usid
    # sortiere: innerhalb jedes usid am aktuellsten zuerst
    df = df.sort_values(["usid", "_time"], ascending=[True, False])
    return df


def show_top_reddit_posts_per_article(df: pd.DataFrame, articles: list[dict], n: int = 20, exact_written_20: bool = True):
    """
    Prints N posts per article (usid). Uses articles list to show ORF title too.
    If exact_written_20=True: shows the last N points by time (tail), which better matches "what was written".
    """
    if df.empty:
        print("DataFrame is empty; nothing to show.")
        return

    # map usid -> orf title
    usid_to_orf_title = {a.get("usid"): (a.get("title") or "") for a in articles if a.get("usid")}

    # keep group order stable
    # behalte Gruppenreihenfolge stabil
    for usid, grp in df.groupby("usid", sort=False):
        orf_title = usid_to_orf_title.get(usid, "")

        print("\n" + "=" * 120)
        print(f"USID: {usid}")
        if orf_title:
            print(f"ORF:  {orf_title[:200]}")
        print(f"Found points in DB: {len(grp)} | showing {min(n, len(grp))}")
        print("-" * 120)

        if exact_written_20:
            # show the last N by write time (in case more than N exist for this usid)
            # zeige die letzten N nach Schreibzeit (falls mehr als N für dieses usid existieren)
            top = grp.sort_values("_time", ascending=True).tail(n).sort_values("_time", ascending=True)
        else:
            # show most recent first
            # zeige am aktuellsten zuerst
            top = grp.head(n)

        for i, (_, row) in enumerate(top.iterrows(), 1):
            title = str(row.get("title") or "")
            permalink = str(row.get("permalink") or "")
            url = str(row.get("url") or "")
            subreddit = str(row.get("source") or "")
            tm = row.get("_time")
            words = int(row.get("checked_word_count", 0) or 0)
            groups = int(row.get("group_matches_in_window", 0) or 0)

            print(f"[{i:02d}] {title[:180]}")
            print(f"     subreddit: {subreddit} | words={words} | groups={groups} | time={tm}")
            print(f"     permalink: {permalink}")
            print(f"     url:       {url}")

            st = row.get("selftext") or ""
            if st:
                st2 = str(st).replace("\n", " ").strip()
                print(f"     selftext:  {st2[:240]}{'…' if len(st2) > 240 else ''}")
            print()


# ---- run inspection ----
df_reddit = load_reddit_posts_from_influx(lookback="30d")
print("Loaded reddit_post points:", len(df_reddit))

# Shows the last 20 written per usid (recommended)
# Zeigt die letzten 20 geschriebenen pro usid (empfohlen)
show_top_reddit_posts_per_article(df_reddit, articles, n=20, exact_written_20=True)

# Optional: interactive peek
# Optional: interaktiver Einblick
df_reddit.head()


Loaded reddit_post points: 81

USID: news:3416950
ORF:  Moldawiens Präsidentin für Vereinigung mit Rumänien
Found points in DB: 3 | showing 3
------------------------------------------------------------------------------------------------------------------------
[01] Radfahrer zu töten reicht nicht für unbedingte Haft dazu muss man wohl auch angetrunken sein.
     subreddit: Fahrrad | words=36 | groups=0 | time=2025-12-22 11:37:35+00:00
     permalink: https://www.reddit.com/r/Fahrrad/comments/1psxgv2/radfahrer_zu_töten_reicht_nicht_für_unbedingte/
     url:       https://www.reddit.com/r/Fahrrad/comments/1psxgv2/radfahrer_zu_töten_reicht_nicht_für_unbedingte/
 Landwirt überrollt und tötet Radfahrer am Radweg mit Traktor 1800€ und 5 Monate bedingte Haft.

[02] Moldawiens Präsidentin für Vereinigung mit Rumänien
     subreddit: de | words=6 | groups=0 | time=2026-01-12 19:58:58+00:00
     permalink: https://www.reddit.com/r/de/comments/1qb5far/moldawiens_präsidentin_für_vereinigung_mit/

,_time,usid,source,reddit_id,title,permalink,url,checked_word_count,group_matches_in_window,selftext
0,2026-01-13 08:27:14+00:00,news:3416950,Austria,1qbm0o4,Zahn-Füllungen sind mittlerweile seit über ein...,https://www.reddit.com/r/Austria/comments/1qbm...,https://www.reddit.com/r/Austria/comments/1qbm...,338,0,Was wurde eigentlich aus den Verhandlungen zwi...
54,2026-01-12 19:58:58+00:00,news:3416950,de,1qb5far,Moldawiens Präsidentin für Vereinigung mit Rum...,https://www.reddit.com/r/de/comments/1qb5far/m...,https://orf.at/stories/3416950/,6,0,None
13,2025-12-22 11:37:35+00:00,news:3416950,Fahrrad,1psxgv2,Radfahrer zu töten reicht nicht für unbedingte...,https://www.reddit.com/r/Fahrrad/comments/1psx...,https://www.reddit.com/r/Fahrrad/comments/1psx...,36,0,https://kaernten.orf.at/stories/3335027/\r\n\r...
71,2026-01-13 11:32:27.469573+00:00,news:3416953,search_de,1qbjm3m,"100,000 😊",https://www.reddit.com/r/BoltEV/comments/1qbjm...,https://i.redd.it/givumipd42dg1.jpeg,8,3,None
70,2026-01-13 11:32:27.468576+00:00,news:3416953,search_de,1qbnvuy,Damay mga estudyante! Trump nagkansela ng higi...,https://www.reddit.com/r/newsPH/comments/1qbnv...,https://i.redd.it/y0718eenf3dg1.jpeg,39,4,"Mahigit 100,000 visa na ang binawi ng US State..."


## Technisches Fazit

- RSS statt HTML → stabil & sauber
- InfluxDB als Speicher **und** Pipeline-State
- heuristisches Matching statt Blackbox-NLP
- vollständig reproduzierbar im Notebook
